In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q accelerate==0.28.0
!pip install -U -q datasets==2.18.0
# !pip install -U -q peft==0.10.0
!pip install -U -q bitsandbytes==0.43.1
!pip install -U -q trl==0.8.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
distributed 2024.5.1 requires dask==2024.5.1, but you have dask 2024.7.0 which is incompatible.
gcsfs 2024.5.0 requires fsspec==2024.5.0, but you have fsspec 2024.2.0 which is incompatible.
rapids-dask-dependency 24.6.0a0 requires dask==2024.5.1, but you have dask 2024.7.0 which is incompatible.
s3fs 2024.5.0 requires fsspec==2024.5.0.*, but you have fsspec 2024.2.0 which is incompatible.
ERROR:

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tuning HuggingFace SmolLM-135M-Instruct with ultrafeedback"
os.environ["WANDB_NAME"] = "ft-smollm-135M-instruct-on-hf-ultrafeedback"
os.environ["MODEL_NAME"] = "HuggingFaceTB/SmolLM-135M-Instruct"
os.environ["TOKENIZER_NAME"] = "HuggingFaceTB/SmolLM-135M-Instruct"
os.environ["DATASET"] = "HuggingFaceH4/ultrafeedback_binarized"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

ds=load_dataset(os.getenv("DATASET"), split=["train_prefs","test_prefs"])
ds

Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

[Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 61135
 }),
 Dataset({
     features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
     num_rows: 2000
 })]

In [5]:
train_ds=ds[0].shuffle(seed=42).select(range(30000))
eval_ds=ds[1].shuffle(seed=42).select(range(1000))

print(train_ds)
print(eval_ds)

Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 30000
})
Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 1000
})


In [6]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("TOKENIZER_NAME"), add_eos_token=True)

# TODO, need to invesgate the model architecture
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="left"

tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

In [7]:
import torch, multiprocessing

def preprocess(x):
    x["chosen"]=tokenizer.apply_chat_template(x["chosen"], tokenize=False)
    x["rejected"]=tokenizer.apply_chat_template(x["rejected"], tokenize=False)
    return x

train_ds=train_ds.map(preprocess, num_proc=multiprocessing.cpu_count(), load_from_cache_file=False)
eval_ds=eval_ds.map(preprocess, num_proc=multiprocessing.cpu_count(), load_from_cache_file=False)

Map (num_proc=4):   0%|          | 0/30000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForCausalLM
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("MODEL_NAME"),
    torch_dtype=torch.float16,
#     device_map={"": 0},
    device_map="cuda",
    trust_remote_code=True
)

model.gradient_checkpointing_enable()
model.device

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [9]:
from trl import ORPOTrainer, ORPOConfig

orpo_config=ORPOConfig(
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="steps",
    do_eval=True,
    optim="adamw_8bit",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=4,
    log_level="debug",
    logging_steps=100,
    learning_rate=3e-4,
    eval_steps=100,
    save_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    beta=0.1, # beta is ORPO's lambda
    max_length=1024,
    report_to="tensorboard",
    run_name=os.getenv('WANDB_NAME')
)

trainer = ORPOTrainer(
        model=model,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        args=orpo_config,
        tokenizer=tokenizer,
)

trainer.train()

2024-08-16 00:57:13.991472: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 00:57:13.991592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 00:57:14.184435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3109 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Currently training with a batch size of: 4
***** Running training *****
  Num examples = 30,000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 3,750
  Number of trainable parameters = 134,515,008
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
100,2.213500,1.126738,125.862700,7.945000,1.986000,-0.130048,-0.130186,0.465000,0.000137,-1.301856,-1.300481,19.203047,19.101301,1.052472,-0.742662,0.008684
200,1.167700,1.123410,125.992300,7.937000,1.984000,-0.128094,-0.128028,0.467000,-0.000067,-1.280275,-1.280941,27.744438,27.623596,1.048127,-0.752827,0.012657
300,1.167600,1.130243,126.019300,7.935000,1.984000,-0.129078,-0.128648,0.466000,-0.000430,-1.286477,-1.290777,30.961571,30.864590,1.054378,-0.758650,0.010223
400,1.133000,1.153756,126.112900,7.929000,1.982000,-0.132228,-0.131452,0.451000,-0.000777,-1.314517,-1.322285,33.423428,33.343773,1.077177,-0.765791,0.009335
500,1.164200,1.138232,126.216700,7.923000,1.981000,-0.131492,-0.130917,0.452000,-0.000575,-1.309167,-1.314919,34.655724,34.567570,1.062307,-0.759259,0.009891
600,1.131500,1.139179,126.245500,7.921000,1.980000,-0.131543,-0.130633,0.456000,-0.000910,-1.306329,-1.315426,36.807312,36.689426,1.062788,-0.763908,0.006581
700,1.156400,1.132295,126.284400,7.919000,1.980000,-0.131264,-0.130727,0.471000,-0.000538,-1.307266,-1.312642,38.208782,38.044609,1.056535,-0.757602,0.011177
800,1.156200,1.130976,126.309700,7.917000,1.979000,-0.131362,-0.131329,0.464000,-0.000033,-1.313290,-1.313615,40.047363,39.823219,1.055382,-0.755938,0.025214
900,1.166500,1.121978,126.275200,7.919000,1.980000,-0.130691,-0.130133,0.457000,-0.000558,-1.301330,-1.306914,40.696991,40.511833,1.046179,-0.757998,0.012584
1000,1.171300,1.132857,126.342800,7.915000,1.979000,-0.131463,-0.130934,0.458000,-0.000529,-1.309345,-1.314631,42.355400,42.152828,1.056525,-0.763317,0.018437


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
***** Running Evalua

TrainOutput(global_step=3750, training_loss=1.1452501291910808, metrics={'train_runtime': 19257.7343, 'train_samples_per_second': 1.558, 'train_steps_per_second': 0.195, 'total_flos': 0.0, 'train_loss': 1.1452501291910808, 'epoch': 1.0})

In [10]:
kwargs={
    'model_name': os.getenv("WANDB_NAME"),
    'finetuned_from': os.getenv('MODEL_NAME'),
#     'tasks': 'Text-Generation',
#     'dataset_tags':'',
    'dataset': os.getenv("DATASET")
}

tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(**kwargs)

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

tokenizer config file saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/tokenizer_config.json
Special tokens file saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/special_tokens_map.json
Uploading the following files to aisuko/ft-smollm-135M-instruct-on-hf-ultrafeedback: tokenizer_config.json,vocab.json,merges.txt,README.md,special_tokens_map.json,tokenizer.json
Saving model checkpoint to ft-smollm-135M-instruct-on-hf-ultrafeedback
Configuration saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/config.json
Configuration saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/generation_config.json
Model weights saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/model.safetensors
tokenizer config file saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/tokenizer_config.json
Special tokens file saved in ft-smollm-135M-instruct-on-hf-ultrafeedback/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Languag

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1723770052.1dd0517e0b3d.25.0:   0%|          | 0.00/70.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aisuko/ft-smollm-135M-instruct-on-hf-ultrafeedback/commit/29428ff130f1e75bafa833563f7a6dc94b470933', commit_message='End of training', commit_description='', oid='29428ff130f1e75bafa833563f7a6dc94b470933', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("WANDB_NAME"), 
    torch_dtype=torch.float16, 
    device_map="cuda", 
    trust_remote_code=True
)

chat=[
    [{"role":"user","content":"How is vanilla cultivated?"}],
    [{"role": "user", "content": "How much money do I have if I have one dollar?"}],
    [{"role": "user", "content": "Where is Berlin?"}],
    [{"role": "user", "content": "Give me a list of 5 European countries."}],
    [{"role": "user", "content": "What is AI?"}],
    [{"role": "user", "content": "What can you do right? Exactly?"}]
]


for c in chat:
    p=tokenizer.apply_chat_template(c, tokenize=False)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, do_sample=True, pad_token_id=tokenizer.eos_token_id, top_p=0.9, max_new_tokens=150)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(result)

loading configuration file ft-smollm-135M-instruct-on-hf-ultrafeedback/config.json
Model config LlamaConfig {
  "_name_or_path": "ft-smollm-135M-instruct-on-hf-ultrafeedback",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 49152
}

loading weights file ft-smollm-135M-instruct-on-hf-ultrafeedback/model.safetensors
Instantiating LlamaForCausalLM model under default dtype torch.float

user
How is vanilla cultivated?
assistant
Monet's painting of his favorite painting in 1825, "Le chat l'ôra" ("The horse's car"), has been described as "vanilla" by some. The term "vanilla" originates from the Spanish word "vaillano," meaning "mood" or "aura."
user
How much money do I have if I have one dollar?
assistant
The question you've asked is an interesting one. However, there are several factors that can impact how much you have in a financial transaction. Here's a possible answer:

If you have a single dollar, the total amount you're receiving is simply the amount you have as a single dollar. This is because the dollar is not divisible, and you can only receive as much as you own.

However, if you have multiple dollars, the total amount you're receiving will depend on how many you have. If you have a few dollars, you might receive more than you own, and if you have a few hundred dollars, you might receive more than you have in each dollar.

So, in this case
user
Where is Berli

# Helpful papers

* https://arxiv.org/abs/2403.07691